# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2023 &ndash; Week 2 &ndash; ETH Zurich</center>

## Exercise 1: Storage devices (Optional)

In this exercise, we want to understand the differences between [SSD](https://en.wikipedia.org/wiki/Solid-state_drive), [HDD](https://en.wikipedia.org/wiki/Hard_disk_drive), and [SDRAM](https://en.wikipedia.org/wiki/Synchronous_dynamic_random-access_memory) in terms of __capacity__, __speed__ and __price__. 

### Task 1
Fill in the table below by visiting your local online hardware store and choosing the storage device with largest capacity available but optimizing for read/write speed.
For instance, you can visit Digitec.ch to explore the prices on [SSDs](https://www.digitec.ch/en/s1/producttype/ssd-545?tagIds=76), [HDDs](https://www.digitec.ch/en/s1/producttype/hard-drives-36?tagIds=76), and 
[SDRAMs](https://www.digitec.ch/en/s1/producttype/memory-2?tagIds=76). 
You are free to use any other website for filling the table. 


| Storage Device | Maximum capacity, GB | Price, CHF/GB  | Read speed, GB/s | Write speed, GB/s | Link |
| --------------:| --------------------:| --------------:|-----------------:|------------------:|------|
| HDD            |                      |                |                  |                   |&nbsp;|
| SSD            |                      |                |                  |                   |&nbsp;|
| DRAM           |                      |                |                  |                   |&nbsp;|


### Task 2
Answer the following questions:
1. What type of storage devices above is the cheapest one?
2. What type of storage devices above is the fastest in terms of read speed?

# Exercise 2. KeyValue Vector Clocks

As pointed out in the lecture, the concepts are clearly explained in the Dynamo paper by the DeCandia, G., et. al. (2007). "Dynamo: Amazon’s Highly Available Key-value Store". In SOSP ’07 (Vol. 41, p. 205). [DOI](https://dl.acm.org/citation.cfm?doid=1294261.1294281)

## Task 1
Multiple distributed hash tables use vector clocks for capturing causality among different versions of the same object. In Amazon's Dynamo, a vector clock is associated with every version of every object.

Let $VC$ be an $N$-element array which contains non-negative integers, initialized to 0, representing $N$ logical clocks of the $N$ processes (nodes) of the system. $VC$ gets its $j$ element incremented by one everytime node $j$ performs a write operation on it. <br>
Moreover, $VC(x)$ denotes the vector clock of a write event, and $VC(x)_z$ denotes the element of that clock for the node $z$.

Try to __formally define__ the partial ordering that we get from using vector clocks.

## Task 2

Vector clock antisymmetry property is defined as follows:

If $ VC(x) \lt VC(y)$, then $ ¬ \ (VC(y) \lt VC(x)) $

Prove this property.

## Task 3

Consider $j$ servers in a cluster where $S_j$ denotes the $j$th node.  
In this exercise, we adopt a slightly modified notation from the Dynamo paper:  
- The Dynamo paper indicates the writing server on the edge, we however write it before the colon.  
- For brevity, we index server by position and omit server name in the vector clock.

For example **aa ([$S_0$,0],[$S_1$,4])** with $S_1$ as writing server become **$S_1$ : aa ([0,4])**
So, given the following version evolution DAG for a particular object, complete the vector clocks computed at the corresponding version.

<img src="https://polybox.ethz.ch/index.php/s/iRONxqhpQkRdLeY/download" width=400/>

<img src="https://polybox.ethz.ch/index.php/s/WzJlMxIrA2RGcKh/download" width=400/>

<img src="https://polybox.ethz.ch/index.php/s/nZ83Jb7mrr0uhi8/download" width=400/>

## Task 4

When a get request comes in to Amazon Dynamo with some key, then:
  - The coordinator node (selected from the preference list as the top node for this key) is taking care of this request
  - The coordinator node requests from other nodes (itself + the next N-1 healthy ones on the preference list), and receives, a set of versions for the value associated with the key, that are modelled as __value (vector clock)__ pairs such as a ([1, 3, 2])

### Task 4.1
Given the following list of versions, draw the version DAG that the coordinator node will build for returning available versions.

1 ([0,0,1])  
1 ([0,1,1])  
2 ([1,1,1])  
3 ([0,2,1])  
10 ([1,3,1])

### Task 4.2
Given the following list of versions, draw the version DAG that the coordinator node will build for returning the correct version.


 a ([1,0,0])  
 b ([0,1,0])  
 c ([2,1,0])   
 d ([2,1,1])   
 e ([3,1,1])  
 f ([2,2,1])   
 g ([3,1,2])   
 h ([3,2,3])  
 i ([4,2,2])   
 j ([5,2,2])  
 k ([4,3,3])  
 l ([5,2,3])  
 m ([5,4,3])  
 n ([6,3,3])  
 o ([6,4,4])  

### Task 4.3
Given the following list of versions, draw the version DAG that the coordinator node will build for returning the correct version.

a ([1,0,0,0])  
b ([0,0,0,1])  
aa ([0,0,1,0])  
bb ([0,1,0,0])  
c ([1,2,0,1])  
cc ([0,1,1,2])  
d ([1,3,0,1])  
f ([1,2,1,3])  
e ([2,1,1,2])  
g ([2,2,2,3])  

## Task 5 (Optional)

Consider $j$ servers in a cluster where $S_j$ denotes the $j$th node. The following table denotes the execution of a series of get/put operations. Also each line of the table represents the events that happen at the time time. For example, at time 0 (`t0`) servers $S_1$ and $S_3$ perform operations. Moreover, when reading and writing an object, we are provided with / must provide a context respecitvely. The context itself is the vector clock, and helps the routines understand what version of the object they are dealing with, and what the new, updated version of the context will be.

For the `get` and `put` routines, we have the following signatures:

* `get(key)` $\rightarrow$ `[val_1, val_2, ...]`, $C_{key}$ `(` $VC$ `(key))` 
  * Example: `get("foo")` $\rightarrow$ `[bar_1, bar_2]`, $C_{2}$ `([1, 0, 1, 0]) # We assume the existence of 4 nodes` 

* `put(key, context, val)` $\rightarrow$ `None`
  * Example: `put("foo",` $C_2$`, "bar")`  

Note that the $C_{key}$ elements are just notation, and are meant to highlight that the context gets passed around between the `get` and `put` routines in a real API. 

Complete missing `[list_values],` $C_{key}$ `([vector_clock])` tuples for the calls below.

<table>
  <tr><th></th><th>S0</th><th>S1</th><th>S2</th><th>S3</th><th>S4</th></tr>
  <tr>
    <td>t0</td>
    <td></td>
    <td>Get(1)$\rightarrow$ _______________, $C_{1}$(_______________)<br>Put(1, _____, ”a”)</td>
    <td></td>
    <td>Get(1)$\rightarrow$ _______________, $C_{2}$(_______________)<br>Put(1, _____, ”bb”)</td>
    <td></td>
  </tr>
  <tr>
    <td>t2</td>
    <td>Get(1)$\rightarrow$ _______________, $C_4$(_______________)<br>Put (1, _____, “rr”)</td>
    <td>Get(1)$\rightarrow$ _______________, $C_5$(_______________)<br>Put (1, _____, ”dd”)
    <td></td>
    <td></td>
    <td></td>
    <td></td>
  </tr>
  <tr>
    <td>t4</td>
    <td></td>
    <td></td>
    <td>Get(1)$\rightarrow$ _______________, $C_9$(_______________) <br>Put(1, _____, ”ccc”)</td>
    <td>Get(1)$\rightarrow$ _______________, $C_{10}$(_______________) <br> Put(1, _____, ”dd”)</td>
    <td></td>
  </tr>
  <tr>
    <td>t5</td>
    <td></td>
    <td></td>
    <td></td>
    <td></td>
    <td>Get(1)$\rightarrow$ _______________, $C_{11}$(_______________) <br>Put(1, _____, “fff”)</td>
  </tr>
</table>

The DAG below shows the interaction among nodes when retrieving values. You can use it for determining the expected values.

<img src="https://polybox.ethz.ch/index.php/s/YoWi7QK2DcMHJFe/download" width=400/>

# Exercise 3. Merkle Trees
A hash tree or Merkle tree is a binary tree in which every leaf node gets as its label a data block and every non-leaf node is labelled with the cryptographic hash of the labels of its child nodes. 

Some KeyValue stores use Merkle trees for efficiently detecting inconsistencies in data between replicas. 

This works by exchaging first the root hash, comparing it with their own. If the hashes match, the replicas are synchronised. If they do not match, then the children of the node (in the Merkle tree) will be retrieved, and their hashes will be compared. This process continues until the inconsistent leave(s) are identified. 

## Task 1
The two pictures below depict two Merkle trees each one belonging to two different replicas. Both should represent the same object.

For the two pairs of trees below. Specify if it is a possible configuration as well as which nodes have to be exchanged in order to sync the trees, if applicable.  

<img src="https://polybox.ethz.ch/index.php/s/vqj7AOAozZKEO3N/download" width=800/>

## Task 2
Repeat the exercise for the following pair of Merkle Trees.

<img src="https://polybox.ethz.ch/index.php/s/TwFd3KDxTrqq2B1/download" width=800/>

# Exercise 4. Virtual nodes

Virtual nodes were introduced to avoid assigning data in an unbalanced manner and coping with hardware heterogeneity by taking into consideration the physical capacity of each server

Let assume we have ten servers ($i_1$ to $i_{10}$) each with the following amount of main memory: `8GiB, 16GiB, 32GiB, 8GiB, 16GiB, 0.5TiB, 1TiB, 0.25TiB, 10GiB, 20GiB`. Calculate the number of virtual nodes/tokens each server should get according to its main memory capacity if we want to have a total of `256` virtual nodes/tokens.

Just for the purpose of the exercises if you get a fractional number of virtual nodes, always round up, even if the total sum of nodes in the end exceeds `256`.

# Exercise 5. Overview of JSONiq

We give a gentle introduction to JSONiq and its arithmetic and logical operators. JSONiq is going to accompany you throughout the course as we will introduce more complex operations. To get started, you first need to execute the cell below to activate the RumbleDB magic (you do not need to understand what it does, this is just initialization Python code).

In [ ]:
!pip install rumbledb
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://localhost:9090/jsoniq


Now we are all set! You can now start reading and executing the JSONiq queries as you go, and you can even edit them!

## JSON

As explained on the [official JSON Web site](http://www.json.org/), JSON is a lightweight data-interchange format designed for humans as well as for computers. It supports as values:
- objects (string-to-value maps)
- arrays (ordered sequences of values)
- strings
- numbers
- booleans (true, false)
- null

JSONiq provides declarative querying and updating capabilities on JSON data. You can refer to [the documentation of JSONiq](https://www.jsoniq.org/docs/JSONiq/webhelp/index.html) for more detail.

## Elevator Pitch

JSONiq is based on XQuery, which is a W3C standard (like XML and HTML). XQuery is a very powerful declarative language that originally manipulates XML data, but it turns out that it is also a very good fit for manipulating JSON natively.
JSONiq, since it extends XQuery, is a very powerful general-purpose declarative programming language. Our experience is that, for the same task, you will probably write about 80% less code compared to imperative languages like JavaScript, Python or Ruby. Additionally, you get the benefits of strong type checking without actually having to write type declarations.
Here is an appetizer before we start the tutorial from scratch.


In [ ]:
%%jsoniq

let $stores :=
[
  { "store number" : 1, "state" : "MA" },
  { "store number" : 2, "state" : "MA" },
  { "store number" : 3, "state" : "CA" },
  { "store number" : 4, "state" : "CA" }
]
let $sales := [
   { "product" : "broiler", "store number" : 1, "quantity" : 20  },
   { "product" : "toaster", "store number" : 2, "quantity" : 100 },
   { "product" : "toaster", "store number" : 2, "quantity" : 50 },
   { "product" : "toaster", "store number" : 3, "quantity" : 50 },
   { "product" : "blender", "store number" : 3, "quantity" : 100 },
   { "product" : "blender", "store number" : 3, "quantity" : 150 },
   { "product" : "socks", "store number" : 1, "quantity" : 500 },
   { "product" : "socks", "store number" : 2, "quantity" : 10 },
   { "product" : "shirt", "store number" : 3, "quantity" : 10 }
]
let $join :=
  for $store in $stores[], $sale in $sales[]
  where $store."store number" = $sale."store number"
  return {
    "nb" : $store."store number",
    "state" : $store.state,
    "sold" : $sale.product
  }
return [$join]



## All JSON values are JSONiq, too

The first thing you need to know is that a well-formed JSON document is a JSONiq expression as well.
This means that you can copy-and-paste any JSON document into a query. The following are JSONiq queries that are "idempotent" (they just output themselves):

In [ ]:
%%jsoniq
{ "pi" : 3.14, "sq2" : 1.4 }

In [ ]:
%%jsoniq
[ 2, 3, 5, 7, 11, 13 ]

In [ ]:
%%jsoniq
{
      "operations" : [
        { "binary" : [ "and", "or"] },
        { "unary" : ["not"] }
      ],
      "bits" : [
        0, 1
      ]
    }

In [ ]:
%%jsoniq
[ { "Question" : "Ultimate" }, ["Life", "the universe", "and everything"] ]

This works with objects, arrays (even nested), strings, numbers, booleans, null.

It also works the other way round: if your query outputs an object, you can use it as a JSON document.
JSONiq is a declarative language. This means that you only need to say what you want - the compiler will take care of the how. 

In the above queries, you are basically saying: I want to output this JSON content, and here it is.

In fact JSONiq makes JSON "dynamic": try to replace numbers with arithmetic formulas, keys with concatenations of strings, etc and see how the resulting JSON object is automatically created.

In [ ]:
%%jsoniq
{
    "foo" : 2 + 2,
    "foo" || "bar" : if(2 gt 1) then true else false
}

# Variables

Some queries involve several chained lookups and function calls. It can become complex.

In [ ]:
%%jsoniq
count(distinct-values(json-file("http://www.rumbledb.org/samples/git-archive-small.json").actor.login))

It is then a natural thing to use variables to store intermediate results. This can be achieved with a series of let clauses. The final result is then put in a return clause.

In [ ]:
%%jsoniq
let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
let $events := json-file($path)
let $actors := $events.actor
let $logins := $actors.login
let $distinct-logins := distinct-values($logins)
return count($distinct-logins)

Note that types are not needed, however they exist! It is possible to add a static type to each variable.
Since values can be sequences, you can add suffixes for cardinality: `*` for a sequence of arbitrary length, `?` for zero or one item, `+` for one or more items.

In [ ]:
%%jsoniq
let $path as string := "http://www.rumbledb.org/samples/git-archive-small.json"
let $events as object* := json-file($path)
let $actors as object* := $events.actor
let $logins as string* := $actors.login
let $distinct-logins as string* := distinct-values($logins)
let $count as integer := count($distinct-logins)
return $count

As you can see, variables can be used to store single items, as well as enormous sequences. RumbleDB will automatically select the best way to evaluate your query.

Note that it is possible to reuse variable names. However, these are not assignments: these are bindings. Reusing a variable name hides the previous binding.

In [ ]:
%%jsoniq
let $v as string := "http://www.rumbledb.org/samples/git-archive-small.json"
let $v as object* := json-file($v)
let $v as object* := $v.actor
let $v as string* := $v.login
let $v as string* := distinct-values($v)
let $v as integer := count($v)
return $v

## Iteration

It is possible to iterate on the elements in a sequence, like so:

In [ ]:
%%jsoniq
for $i in 1 to 10
return $i * 2

The sequence to iterator on can itself come from a dataset, such as the one we were using previously:

In [ ]:
%%jsoniq
for $event in json-file("http://www.rumbledb.org/samples/git-archive-small.json")
return size($event.payload.commits)

`for` clauses can be mixed with `let` clauses:

In [ ]:
%%jsoniq
let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
for $event in json-file($path)
let $commits := $event.payload.commits
return size($commits)

And the results can also be nested in a more complex query: for example, let us compute the max of all these array sizes.

In [ ]:
%%jsoniq
max(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  return size($commits)
)

A third kind of clause is the `where` clause: it allows you to filter events. Let us only keep those with more than 10 commits, and count them.

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) gt 10
  return $event
)

## Simple calculations

Let us now look closer arithmetics, comparison and logic expressions. They are particularly useful in a where clause or in a Boolean predicate, however these expressions can be used just about anywhere as this is a functional language.

### Arithmetics

JSONiq works like a calculator and can do arithmetics with the four basic operations.

In [ ]:
%%jsoniq
 (38 + 2) div 2 + 11 * 2


(mind the division operator which is the "div" keyword. The slash operator has different semantics).

Like JSON, JSONiq works with decimals and doubles:

In [ ]:
%%jsoniq
 6.022e23 * 42

JSONiq also support modulos, integer division, and has a rich function library (trigonometry, logarithms, exponential, powers, etc).

## Comparison

Values (numbers, strings, dates, etc) can be compared with the binary operators `eq`, `ne`, `gt`, `ge`, `lt` and `le`.
Let us change the comparison used in the where clause with other kinds.

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) gt 10
  return $event
)

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) eq 10
  return $event
)

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) ne 10
  return $event
)

In [ ]:
%%jsoniq
count(
  let $path := "http://www.rumbledb.org/samples/git-archive-small.json"
  for $event in json-file($path)
  let $commits := $event.payload.commits
  where size($commits) le 10
  return $event
)

Why not = or < or >=? This is because these are more powerful. In fact, they implicitly perform an existential quantification over the operands.

In [ ]:
%%jsoniq
1 to 10 = 5

In [ ]:
%%jsoniq
1 to 10 > 11 to 20

### Logical operations

JSONiq supports Boolean logic.

In [ ]:
%%jsoniq
true and false

In [ ]:
%%jsoniq
(true or false) and (false or true)

The unary not is also available:

In [ ]:
%%jsoniq
not true

Note that JSONiq, unlike SQL, does two-valued logic. Nulls are automatically converted to false.

In [ ]:
%%jsoniq
null and true

Some non-Booleans can also get converted. For example, non-empty strings are converted to true and empty strings to false.

In [ ]:
%%jsoniq
not ""

In [ ]:
%%jsoniq
not "non empty"

Zero is converted to false, non-zero numbers to true.

In [ ]:
%%jsoniq
not 0

In [ ]:
%%jsoniq
not 1e10

### Strings

JSONiq is capable of manipulating strings as well, using functions:


In [ ]:
%%jsoniq
concat("Hello ", "Captain ", "Kirk")

In [ ]:
%%jsoniq
substring("Mister Spock", 8, 5)

JSONiq comes up with a rich string function library out of the box, inherited from its base language. These functions are listed [here](https://www.w3.org/TR/xpath-functions-30/) (actually, you will find many more for numbers, dates, etc).



### Sequences

Until now, we have only been working with single values (an object, an array, a number, a string, a boolean). JSONiq supports sequences of values. You can build a sequence using commas:


In [ ]:
%%jsoniq
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [ ]:
%%jsoniq
1, true, 4.2e1, "Life"

The "to" operator is very convenient, too:

In [ ]:
%%jsoniq
 (1 to 100)

Some functions even work on sequences:

In [ ]:
%%jsoniq
sum(1 to 100)

In [ ]:
%%jsoniq
string-join(("These", "are", "some", "words"), "-")

In [ ]:
%%jsoniq
count(10 to 20)

In [ ]:
%%jsoniq
avg(1 to 100)

Unlike arrays, sequences are flat. The sequence (3) is identical to the integer 3, and (1, (2, 3)) is identical to (1, 2, 3). See [this post](https://www.jsoniq.org/docs/Introduction_to_JSONiq/html/ch17.html#:~:text=Any%20value%20returned%20by%20or,them%20to%20form%20bigger%20sequences.) for more detail on sequences vs. arrays.

and even filter out some values:

In [ ]:
%%jsoniq
let $sequence := 1 to 10
for $value in $sequence
let $square := $value * $value
where $square < 10
return $square

Note that you can only iterate over sequences, not arrays. To iterate over an array, you can obtain the sequence of its values with the `[]` operator, like so:


In [ ]:
%%jsoniq
[1, 2, 3][]

Compare the following two queries:

In [ ]:
%%jsoniq
let $sequence := [1, 2, 3][]
for $value in $sequence
let $square := $value * $value
where $square < 10
return $square

In [ ]:
%%jsoniq
let $sequence := [1, 2, 3]
for $value in $sequence
let $square := $value * $value
where $square < 10
return $square

### Conditions

You can make the output depend on a condition with an `if-then-else` construct:

In [ ]:
%%jsoniq
for $x in 1 to 10
return if ($x < 5) then $x
                   else -$x

Note that the `else` clause is required - however, it can be the empty sequence `()` which is often when you need if only the `then` clause is relevant to you. For instance,

In [ ]:
%%jsoniq
for $x in 1 to 10
return if ($x < 5) then $x
                   else ()

### Composability of Expressions

Now that you know of a couple of elementary JSONiq expressions, you can combine them in more elaborate expressions. For example, you can put any sequence of values in an array:

In [ ]:
%%jsoniq
[ 1 to 10 ]

Or you can dynamically compute the value of object pairs (or their key):

In [ ]:
%%jsoniq
{
      "Greeting" : (let $d := "Mister Spock"
                    return concat("Hello, ", $d)),
      "Farewell" : string-join(("Live", "long", "and", "prosper"),
                               " ")
}

You can dynamically generate object singletons (with a single pair):


In [ ]:
%%jsoniq
{ concat("Integer ", 2) : 2 * 2 }

and then merge lots of them into a new object with the `{| |}` notation:

In [ ]:
%%jsoniq
{|
    for $i in 1 to 10
    return { concat("Square of ", $i) : $i * $i }
|}